In [1]:
import pandas as pd
from scipy.stats import entropy
from collections import defaultdict
from scipy.stats import percentileofscore

def generate_human_histogram_df(responses_df, PROVA, CO_PROVA, o, min_nota, max_nota):

    #responses_df = responses_df[responses_df['CO_PROVA_'+PROVA] == CO_PROVA]
   
    itens_df = pd.read_csv(item_df_path, sep=';', encoding = "latin")
    itens_df = itens_df[itens_df['SG_AREA'] == PROVA]
    itens_df = itens_df[itens_df['TP_LINGUA'] != 0]
    print('len_itens_df', len(itens_df))
    
    itens_df = itens_df.sort_values(by=['CO_PROVA','CO_POSICAO'], ascending=True) 
    
    map_co_item_to_target_item_id = dict()
    target_item_id = 0
    for idx, item in itens_df.iterrows():
        co_item = item['CO_ITEM']
        co_posicao = item['CO_POSICAO']
        co_prova = item['CO_PROVA']
        gabarito = item['TX_GABARITO']

        if co_prova == CO_PROVA:
            target_item_id += 1
            map_co_item_to_target_item_id[co_item] = target_item_id


        #if co_item == 111664 and co_prova:
        #    print('co_item', co_item, 'co_posicao', co_posicao, 'co_prova', co_prova, 'gabarito', gabarito, 'target_item_id', target_item_id)
    
    #print('map_co_item_to_target_item_id', map_co_item_to_target_item_id)
    
    map_co_prova_co_item_to_target_prova_item_id = defaultdict(dict)
    map_co_prova_item_id_to_co_item = defaultdict(dict)
    dic_item_id_count = defaultdict(int)
    for idx, item in itens_df.iterrows():
        co_item = item['CO_ITEM']
        co_posicao = item['CO_POSICAO']
        co_prova = item['CO_PROVA']
        gabarito = item['TX_GABARITO']

        dic_item_id_count[co_prova] += 1

        if co_item not in map_co_item_to_target_item_id:
            #print('co_item', co_item, 'not found in map_co_item_to_target_item_id')
            continue

        #print(co_prova, 'co_posicao', co_posicao, 'item_id', dic_item_id_count[co_prova], gabarito)
        
        #if co_item == 111664:
        #    print('co_item', co_item, 'co_posicao', co_posicao, 'co_prova', co_prova, 'gabarito', gabarito, 'dic_item_id_count[co_prova]', dic_item_id_count[co_prova])
        
        map_co_prova_item_id_to_co_item[co_prova][dic_item_id_count[co_prova]]=co_item
        map_co_prova_co_item_to_target_prova_item_id[co_prova][co_item] = map_co_item_to_target_item_id[co_item]

        #print('co_prova', co_prova, 'co_posicao', co_posicao, 'item', dic_item_id_count[co_prova], 'is',co_item)

    #print('map_co_prova_item_id_to_co_item contains PROVAS', map_co_prova_item_id_to_co_item.keys())
    #for co_prova in map_co_prova_item_id_to_co_item.keys():
        #print('prova ', co_prova, 'contains itens', map_co_prova_item_id_to_co_item[co_prova].keys())

    # ACERTA ITENS_DF
    itens_df = itens_df[itens_df['CO_PROVA'] == CO_PROVA].sort_values(by='CO_POSICAO', ascending=True)
    itens_df.reset_index()
    itens_df['human_item_id'] = range(1, len(itens_df) + 1)
    
    #print('len_responses_df', len(responses_df))
    #print('map_co_prova_item_id_to_co_item', map_co_prova_item_id_to_co_item)
    
    options =  ['A', 'B', 'C', 'D', 'E']
    
    histograms_options = {}
    histograms_correct = { }
    correct_options = list()
    correct_response_pattern = ""

    student_count = 0

    record_count = 0

    filtered_responses_df = responses_df[(responses_df['NU_NOTA'] >= min_nota) & (responses_df['NU_NOTA'] <= max_nota)]
    
    dic_correct_option = dict()
    for index, student in filtered_responses_df.iterrows():      
        #if student['TP_PRESENCA_' + PROVA] != 1:
        #    continue  
    
        co_prova = student['CO_PROVA']

        if co_prova not in dic_item_id_count:
            continue
        
        #if co_prova != CO_PROVA:
        #    continue
        
        student_count += 1
        #if student_count % 10000 == 0:
       #     print('student_count', student_count)

        #if student_count == 200000:
        #    break

        record_count += 1

        for item_id, choice_original_order in enumerate(student['TX_RESPOSTAS'], start=1):
            
            #if co_prova != CO_PROVA:
            #    continue
            
            #shuffled_response_pattern = item['SHUFFLED_ORDER_TX_RESPOSTAS']
            response_pattern = student['TX_RESPOSTAS']
            #shuffled_correct_response_pattern = item['SHUFFLED_ORDER_TX_GABARITO']
            correct_response_pattern = student['TX_GABARITO']
        
            #choice_shuffled_order = shuffled_response_pattern[item_id-1]
            #correct_shuffled_order = shuffled_correct_response_pattern[item_id-1]

            choice_original_order = response_pattern[item_id-1]
            correct_original_order = correct_response_pattern[item_id-1]

            if item_id not in map_co_prova_item_id_to_co_item[co_prova]:
                #print('item_id', item_id, 'not in co_prova', co_prova)
                continue
                
            co_item = map_co_prova_item_id_to_co_item[co_prova][item_id]
            target_item_id = map_co_prova_co_item_to_target_prova_item_id[co_prova][co_item]
            correct = choice_original_order == correct_original_order

            #if co_item == 111664:
            #    print("Adding for co_prova", co_prova, "co_item", co_item, "item_id", item_id, "target_item_id", target_item_id, "correct", correct_original_order)
            #    print("AAAH! correct option for target_item_id " + str(target_item_id) + " differs. " \
            #                    "item_id is",item_id, \
            #                    "co_prova is",co_prova,  \
            #                    "gabarito is", correct_response_pattern)
            
            if target_item_id in dic_correct_option and dic_correct_option[target_item_id] != correct_original_order:
                raise Exception("Fatal error: correct option for target_item_id " + str(target_item_id) + " differs. " \
                                "item_id is",item_id,
                                "co_prova is",co_prova,  \
                                "gabarito is", correct_response_pattern, \
                                "It is " + dic_correct_option[target_item_id] + "but we're trying to change to " + correct_original_order)

            dic_correct_option[target_item_id] = correct_original_order
            
            # Update the histogram for the current position and letter
            if target_item_id not in histograms_options:
                histograms_options[target_item_id] = { }
                for option in options:
                    histograms_options[target_item_id][option] = 0
                histograms_correct[target_item_id] = { }

                        
            if o == 'choice_original_order':
                letter = choice_original_order
                if letter not in options:
                    continue
                histograms_options[target_item_id][letter] = histograms_options[target_item_id].get(letter, 0) + 1
                          
            elif o == 'choice_shuffled_order':
                letter = choice_shuffled_order
                histograms_options[target_item_id][letter] = histograms_options[target_item_id].get(letter, 0) + 1
            
            if choice_original_order == correct_original_order:
                    column_name = 'human_accuracy'
                    histograms_correct[target_item_id][column_name] = histograms_correct[target_item_id].get(column_name, 0) + 1
            else:
                    column_name = 'WRONG'
                    histograms_correct[target_item_id][column_name] = histograms_correct[target_item_id].get(column_name, 0) + 1    

    print('record_count', record_count)
    #print('histogram_correct')
    #for key in  sorted(histograms_correct.keys()):
    #    print(f'{key}: {histograms_correct[key]}')

    #print('dict_correct_option')
    #for key in  sorted(dic_correct_option.keys()):
    #    print(f'{key}: {dic_correct_option[key]}')


    #print('histogram_options', histograms_options)
    # Convert the histograms to a DataFrame for better presentation
    histogram_options_df = pd.DataFrame(histograms_options).sort_index()
    histogram_options_df.fillna(0, inplace=True)
    histogram_options_df = histogram_options_df.sort_index(axis=1)

    histogram_correct_df = pd.DataFrame(histograms_correct).sort_index()
    histogram_correct_df.fillna(0, inplace=True)
    histogram_correct_df = histogram_correct_df.sort_index(axis=1)

    # Display the resulting histogram DataFrame
    normalized_histogram_options_df = histogram_options_df/histogram_options_df.sum()
    normalized_histogram_options_df = normalized_histogram_options_df.transpose()
    normalized_histogram_options_df['human_entropy'] = normalized_histogram_options_df.apply(entropy, axis=1) # +  np.random.uniform(-0.001, 0.001, len(normalized_histogram_df['A']))  
    normalized_histogram_options_df['human_item_id'] = range(1, len(normalized_histogram_options_df) + 1)

    normalized_histogram_correct_df = histogram_correct_df/histogram_correct_df.sum()
    normalized_histogram_correct_df = normalized_histogram_correct_df.transpose()
    normalized_histogram_correct_df['human_item_id'] = range(1, len(normalized_histogram_correct_df) + 1)
   
    normalized_histogram_options_df['CORRECT_OPTION_HUMAN'] =  [value for key, value in sorted(dic_correct_option.items())]
    histogram_df = pd.merge(normalized_histogram_options_df, normalized_histogram_correct_df, how='left', on='human_item_id')
    return pd.merge(histogram_df, itens_df, how='left', on='human_item_id')


     

In [2]:
def generated_sorted_letters_table(results_df, filepath):

    co_posicao_items = list()
    ordered_letters_easy_to_hard = list()
    ordered_letters_hard_to_easy = list()
    correct_options = list()
    for idx, item in results_df.iterrows():
        dic = dict()

        co_posicao_items.append(item['CO_POSICAO'])
        correct_options.append(item['CORRECT_OPTION_HUMAN'])
    
        dic['A'] = item['human_A_75_100']
        dic['B'] = item['human_B_75_100']
        dic['C'] = item['human_C_75_100']
        dic['D'] = item['human_D_75_100']
        dic['E'] = item['human_E_75_100']

        dic = dict(sorted(dic.items(), key=lambda item: item[1], reverse=False))
        letter_order = item['CORRECT_OPTION_HUMAN']
        for letter in dic.keys():
            if letter not in letter_order:
                letter_order += letter
        ordered_letters_easy_to_hard.append(letter_order)

        dic = dict(sorted(dic.items(), key=lambda item: item[1], reverse=True))
        letter_order = item['CORRECT_OPTION_HUMAN']
        for letter in dic.keys():
            if letter not in letter_order:
                letter_order += letter
        ordered_letters_hard_to_easy.append(letter_order)

    df = pd.DataFrame({'CO_POSICAO': co_posicao_items, 'ORDERED_OPTIONS_HARD_TO_EASY': ordered_letters_hard_to_easy, 'ORDER_OPTIONS_EASY_TO_HARD': ordered_letters_easy_to_hard, 'CORRECT_OPTION': correct_options})

    df.to_csv(filepath)

    



In [3]:
import numpy as np
import pandas as pd

# 2022
# CH 1062 
# MT 1082
# CN 1092

# 2021
# CH 886
# CN 916
# MT 906

# 2020
# CH 574
# CN 604
# MT 594

# 2019
# CH 520
# MT 522
# CN 519

# 2018
# CH 464
# MT 466

ch_tuples = [(2022, 'CH', 1062), (2021, 'CH', 886), (2020, 'CH', 574), (2019, 'CH', 520), (2018, 'CH', 464)]
cn_tuples = [(2022, 'CN', 1092), (2021, 'CN', 916), (2020, 'CN', 604), (2019, 'CN', 519), (2018, 'CN', 463)]
mt_tuples = [(2022, 'MT', 1082), (2021, 'MT', 906), (2020, 'MT', 594), (2019, 'MT', 522), (2018, 'MT', 466)]
lc_tuples = [(2022, 'LC', 1072), (2021, 'LC', 896), (2020, 'LC', 584), (2019, 'LC', 521)]


for tuple in ch_tuples + cn_tuples + mt_tuples + lc_tuples:
    print('reading enem...', tuple)
    
    year = tuple[0]
    PROVA = tuple[1]
    co_prova = tuple[2]
    
    columns = ['CO_PROVA_'+PROVA,'NU_NOTA_'+PROVA,'TX_RESPOSTAS_'+PROVA,'TX_GABARITO_'+PROVA, 'TP_PRESENCA_'+PROVA]

    enem_human_df = pd.read_csv(f"C:/Users/pedro/Downloads/TRI/test_responses_humans/{year}/responses/test_responses_humans_{PROVA}_{year}.csv")
    enem_human_df.sort_values(by='NU_NOTA', inplace=True, ascending=False)

    #enem_human_df = pd.read_csv(f"C:/Users/pedro/Downloads/TRI/microdados/microdados_enem_{year}/DADOS/MICRODADOS_ENEM_{year}.csv", usecols=columns, sep=";",  encoding='latin') 
    item_df_path = f"c:/Users/pedro/Downloads/TRI/microdados/microdados_enem_{year}/DADOS/ITENS_PROVA_{year}.csv"

    print('enem read.')
    percentiles = [0, 0.25, 0.50, 0.75, 1.0]
    str_percentiles = [0, 25, 50, 75, 100]
    #percentiles = [0,1]
    #percentiles = [0, 50, 100]
    nota_percentis = []
    percentil_idx = 0
    for p in percentiles:
        nota = enem_human_df['NU_NOTA'].quantile(p)
        print('nota', nota)
        nota_percentis.append(nota)

    print('grouping...')
    grouped = enem_human_df.groupby('CO_PROVA')['NU_NOTA'].count().sort_values(ascending=False)
    print(grouped)

    co_provas = enem_human_df['CO_PROVA'].unique()
    enem_human_df.sort_values(by='NU_NOTA', inplace=True, ascending=False)

    all_dfs = []
    for id_percentil in range(0, len(nota_percentis) - 1):
         min_nota = nota_percentis[id_percentil]
         max_nota = nota_percentis[id_percentil+1]
         string_pct = str(str_percentiles[id_percentil]) + "_" + str(str_percentiles[id_percentil+1])
            
         print(min_nota, max_nota)   
         df = generate_human_histogram_df(enem_human_df, PROVA=PROVA, CO_PROVA=co_prova, o='choice_original_order', min_nota=min_nota,max_nota=max_nota)
         df.rename(columns={'A': 'human_A_'+string_pct, 'B': 'human_B_'+string_pct, 'C': 'human_C_'+string_pct, 'D': 'human_D_'+string_pct, 'E': 'human_E_'+string_pct}, inplace=True)
         df.rename(columns={'human_accuracy':'human_accuracy_'+string_pct},inplace=True)
         df.rename(columns={'human_entropy':'human_entropy_'+string_pct},inplace=True)

         if 'human_A_75_100' not in df.columns:
               df = df.drop('CORRECT_OPTION_HUMAN', axis=1)
               df = df.drop('CO_POSICAO', axis=1)
               df = df.drop('CO_HABILIDADE', axis=1)
               df = df.drop('CO_ITEM', axis=1)
               df = df.drop('NU_PARAM_A', axis=1)
               df = df.drop('NU_PARAM_B', axis=1)
               df = df.drop('NU_PARAM_C', axis=1)
               df = df.drop('TX_MOTIVO_ABAN', axis=1)
               df = df.drop('TP_LINGUA', axis=1)
               df = df.drop('TX_COR', axis=1)

        
         if 'WRONG' in df.columns:
               df.drop(columns='WRONG', inplace=True)
         all_dfs.append(df)
         filename = f"C:/Users/pedro/Downloads/TRI/test_responses_humans/{year}/itens/intermediarios/human_itens_{PROVA}_{int(co_prova)}_{year}_{string_pct}.csv"
         print("SAVING", filename, df.columns)
         df.to_csv(filename)
        
    result_df = pd.concat(all_dfs, axis=1, verify_integrity=False)
    print('final columns', sorted(result_df.columns))
    result_df.to_csv(f"C:/Users/pedro/Downloads/TRI/test_responses_humans/{year}/itens/human_itens_{PROVA}_{int(co_prova)}_{year}.csv")

    generated_sorted_letters_table(result_df, f"C:/Users/pedro/Downloads/TRI/test_responses_humans/{year}/itens/human_itens_ordered_options_{PROVA}_{int(co_prova)}_{year}.csv")

        #df = generate_human_histogram_df(enem_human_df, PROVA=exam, CO_PROVA=co_prova, o='correct_percentage', normalize=True)
        #df.to_csv(f"C:/Users/pedro/Downloads/TRI/test_responses_humans/2022/correct_percentage/correct_percentage_{exam}_{int(co_prova)}.csv")

print("FINISHED.")     


reading enem... (2022, 'CH', 1062)


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.